<a href="https://colab.research.google.com/github/HkrAnis/Python-SDV-SyntheticData-ML/blob/main/220616_Hackathon_Task1_CTGAN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Play an audio when the cell has been executed

In [ ]:
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

### Install SDV (Used for the synthesizer)

In [ ]:
pip install sdv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import packages

In [ ]:
from google.colab import drive
import pandas as pd
from sdv.tabular import CTGAN
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split # Model evaluation
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler # Preprocessing
from sklearn.linear_model import Lasso, Ridge, ElasticNet, RANSACRegressor, SGDRegressor, HuberRegressor, BayesianRidge # Linear models
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor  # Ensemble methods
from xgboost import XGBRegressor, plot_importance # XGBoost
from sklearn.svm import SVR, SVC, LinearSVC  # Support Vector Regression
from sklearn.tree import DecisionTreeRegressor # Decision Tree Regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline # Streaming pipelines
from sklearn.decomposition import KernelPCA, PCA # Dimensionality reduction
from sklearn.feature_selection import SelectFromModel # Dimensionality reduction
from sklearn.model_selection import learning_curve, validation_curve, GridSearchCV # Model evaluation
from sklearn.base import clone # Clone estimator
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Connect to the goolge drive dataset

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import the dataset from the google drive 

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Hackathon/data_with_missing.csv")
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,cityid,ID,Race,Education,tag_truncated,year,country
0,0,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,107000.0,20000.0,10000.0,Unavailable,7392,1,Unavailable,Unavailable,NaN,2017-01-01,US
1,1,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,0.0,0.0,0.0,Unavailable,7419,2,Unavailable,Unavailable,NaN,2017-01-01,US
2,2,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,155000.0,0.0,0.0,Unavailable,11527,3,Unavailable,Unavailable,NaN,2017-01-01,US
3,3,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,157000.0,180000.0,35000.0,Unavailable,7472,7,Unavailable,Unavailable,NaN,2017-01-01,US
4,4,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,0.0,0.0,0.0,Unavailable,7322,9,Unavailable,Unavailable,NaN,2017-01-01,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35319,35319,Amazon,L5,Software Engineer,213000,"Seattle, WA",6.0,4.0,Distributed Systems (Back-End),147500.0,38000.0,35000.0,Unavailable,11527,1843,Unavailable,Unavailable,Distributed Systems (Back-End),2018-01-01,US
35320,35320,eBay,Designer III,Product Designer,172000,"San Jose, CA",4.0,2.0,Design,142000.0,15000.0,14200.0,Unavailable,7422,1958,Unavailable,Unavailable,others,2018-01-01,US
35321,35321,Google,T4,Software Engineer,327000,"Seattle, WA",10.0,1.0,Distributed Systems (Back-End),155000.0,150000.0,22000.0,Unavailable,11527,1973,Unavailable,Unavailable,Distributed Systems (Back-End),2018-01-01,US
35322,35322,Microsoft,62,Software Engineer,237000,"Redmond, WA",2.0,2.0,Full Stack,146900.0,73200.0,16000.0,Unavailable,11521,2037,Unavailable,Unavailable,Full Stack,2018-01-01,US


### Remove Unnamed 0 column 

In [ ]:
df.drop(columns=["Unnamed: 0", "tag_truncated", "totalyearlycompensation"], inplace=True)

### Drop the row from city_id where anonymized is present

In [ ]:
i = df[(df.cityid == 'anonymized')].index
df.drop(i, inplace=True)
df

,company,level,title,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,cityid,ID,Race,Education,year,country
0,Oracle,L3,Product Manager,"Redwood City, CA",1.5,1.5,NaN,107000.0,20000.0,10000.0,Unavailable,7392,1,Unavailable,Unavailable,2017-01-01,US
1,eBay,SE 2,Software Engineer,"San Francisco, CA",5.0,3.0,NaN,0.0,0.0,0.0,Unavailable,7419,2,Unavailable,Unavailable,2017-01-01,US
2,Amazon,L7,Product Manager,"Seattle, WA",8.0,0.0,NaN,155000.0,0.0,0.0,Unavailable,11527,3,Unavailable,Unavailable,2017-01-01,US
3,Apple,M1,Software Engineering Manager,"Sunnyvale, CA",7.0,5.0,NaN,157000.0,180000.0,35000.0,Unavailable,7472,7,Unavailable,Unavailable,2017-01-01,US
4,Microsoft,60,Software Engineer,"Mountain View, CA",5.0,3.0,NaN,0.0,0.0,0.0,Unavailable,7322,9,Unavailable,Unavailable,2017-01-01,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35319,Amazon,L5,Software Engineer,"Seattle, WA",6.0,4.0,Distributed Systems (Back-End),147500.0,38000.0,35000.0,Unavailable,11527,1843,Unavailable,Unavailable,2018-01-01,US
35320,eBay,Designer III,Product Designer,"San Jose, CA",4.0,2.0,Design,142000.0,15000.0,14200.0,Unavailable,7422,1958,Unavailable,Unavailable,2018-01-01,US
35321,Google,T4,Software Engineer,"Seattle, WA",10.0,1.0,Distributed Systems (Back-End),155000.0,150000.0,22000.0,Unavailable,11527,1973,Unavailable,Unavailable,2018-01-01,US
35322,Microsoft,62,Software Engineer,"Redmond, WA",2.0,2.0,Full Stack,146900.0,73200.0,16000.0,Unavailable,11521,2037,Unavailable,Unavailable,2018-01-01,US


### Replace all the tags that have a cout less than 100 with Other

In [ ]:
tag= df["tag"].value_counts()
df["tag"]= np.where(df["tag"].isin(tag.index[tag>100]), df["tag"], "Other")
df["tag"].value_counts()

Distributed Systems (Back-End)    6745
Other                             6023
Full Stack                        5788
API Development (Back-End)        3281
ML / AI                           2937
Web Development (Front-End)       1458
Product                           1048
iOS                                791
DevOps                             743
Data                               737
Security                           721
Networking                         683
Android                            604
Mobile (iOS + Android)             563
Technical                          479
Testing (SDET)                     409
User Experience (UX)               395
Site Reliability (SRE)             360
General                            352
Production                         205
Operations                         189
Consumer                           182
Enterprise                         182
Hardware                           163
ASIC Design                        149
Cloud                    

### Replace all the levels that have a count less than 100 with Other

In [ ]:
level= df["level"].value_counts()
df["level"]= np.where(df["level"].isin(level.index[level>100]), df["level"], "Other")
df["level"].value_counts()

Other     7720
L5        3432
L4        3201
L6        2080
L3        1578
          ... 
Band 7     103
4          103
6          103
5          102
L5A        101
Name: level, Length: 69, dtype: int64

### Unmount from google drive

In [ ]:
# drive.flush_and_unmount()

### Define the function that will prepare the model for ML

In [ ]:
def modelPreperation(dataSet):
  dataSet.dropna(inplace=True)
  dataSet=pd.get_dummies(dataSet, columns=["title","company", "level", "location", "tag", "gender", "Race", "Education", "country"])
  dataSet["year"]=dataSet["year"].str[:4].astype(int)
  return dataSet

### Define a function that would Split the dataset between X and y 

In [ ]:
def splitter(df, Ycol):
    # This function takes a dataset and splits it between the independent columns and the dependent column
    # df is a pandas dataframe
    # Ycol is a string representing the column to split 
    return df.loc[:, df.columns != Ycol], df[Ycol]

### Divide the dataset between train and test

In [ ]:
train = df.sample(frac = 0.8, random_state=7)
test = df.drop(train.index)

### Create the synthesize train set 

1-Define the metadata

In [ ]:
Metadata= {'fields':
           {'company': {'type': 'categorical'},
            'level': {'type': 'categorical'},
            'title': {'type': 'categorical'},
            'totalyearlycompensation': {'type': 'numerical', 'subtype': 'float'},
            'location': {'type': 'categorical'},
            'yearsofexperience': {'type': 'numerical', 'subtype': 'float'},
            'yearsatcompany': {'type': 'numerical', 'subtype': 'float'},
            'tag': {'type': 'categorical'},
            'basesalary': {'type': 'numerical', 'subtype': 'float'},
            'stockgrantvalue': {'type': 'numerical', 'subtype': 'float'},
            'bonus': {'type': 'numerical', 'subtype': 'float'},
            'gender': {'type': 'categorical'},
            'cityid': {'type': 'categorical'},
            'ID': {'type': 'categorical'},
            'Race': {'type': 'categorical'},
            'Education': {'type': 'categorical'},
            'year': {'type': 'categorical'},
            'country': {'type': 'categorical'}
            },
           
           'constraints': [],
           "primary_key": []}

2- Choose the synthesizer model

In [ ]:
model = CTGAN()

3- Fit the chosen model 

In [ ]:
model.fit(train)

/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:282:

4- Create the synthetic dataset

In [ ]:
train_s = model.sample(num_rows=len(train))
train_s

In [ ]:
train_save=train_s.copy()

### Append train_s and df

1- Create a new column in train_s and df to be able to seperate them later on 

In [ ]:
df["Table_number"]=0
train_s["Table_number"]=1 
print(df.columns, train_s.columns)

2- Append train_s and df 

In [ ]:
df_appended= df.append(train_s, ignore_index=True)
df_appended

3- Prepare the dataframe for the machine learning model

In [ ]:
df_appended= modelPreperation(df_appended)
df_appended

4- Split the dataset between train_s and df

In [ ]:
df= df_appended[df_appended["Table_number"]==0]
df

In [ ]:
train_s= df_appended[df_appended["Table_number"]==1]
train_s

5- Remove the table number column

In [ ]:
df.drop(columns=["Table_number"], inplace=True)
train_s.drop(columns=["Table_number"], inplace=True)

### Split df between the train part and the test part

In [ ]:
train = df.sample(frac = 0.8, random_state=7)
test = df.drop(train.index)

### Split train, train_s and test

*   X_train and y_train
*   X_train_s and y_train_s
*   X_test and y_test

In [ ]:
X_train, y_train= splitter(train, "basesalary")
X_train_s, y_train_s= splitter(train_s, "basesalary")
X_test, y_test= splitter(test, "basesalary")

### Score the diffent machine learning models

1- Create a list and assign a random seed

In [ ]:
pipeline=[]
seed=7

2- Create the machine learning models that are going to be used 

In [ ]:
Lasso = Pipeline([('scaler', StandardScaler()), ('Lasso', ElasticNet(random_state=seed))])
Ridge = Pipeline([('scaler', StandardScaler()), ('Ridge', Ridge(random_state=seed, tol=10))])
ExtraTreesRegressor = Pipeline([("Scaler", StandardScaler()),("ET", ExtraTreesRegressor(random_state=seed))])
BaggingRegressor = Pipeline([("Scaler", StandardScaler()),("BR", BaggingRegressor(random_state=seed))])
HuberRegressor = Pipeline([("Scaler", StandardScaler()),("Hub-Reg", HuberRegressor())])
BayesianRidge = Pipeline([("Scaler", StandardScaler()),("BR", BayesianRidge())])
DecisionTreeRegressor = Pipeline([("Scaler", StandardScaler()),("DT_reg", DecisionTreeRegressor())])
KNeighborsRegressor = Pipeline([("Scaler", StandardScaler()),("KNN_reg", KNeighborsRegressor())])
GradientBoostingRegressor =  Pipeline([("Scaler", StandardScaler()),("GBoost-Reg", GradientBoostingRegressor())])
                 


3- Append the list with the models and their names

In [ ]:
pipeline.append(("Lasso",Lasso))
pipeline.append(("Ridge",Ridge))
pipeline.append(("ExtraTreesRegressor",ExtraTreesRegressor)) 
pipeline.append(("BaggingRegressor",BaggingRegressor)) 
pipeline.append(("HuberRegressor",HuberRegressor)) 
pipeline.append(("BayesianRidge",BayesianRidge)) 
pipeline.append(("DecisionTreeRegressor",DecisionTreeRegressor)) 
pipeline.append(("KNeighborsRegressor",KNeighborsRegressor)) 
pipeline.append(("GradientBoostingRegressor",GradientBoostingRegressor)) 

4- Define the parameters of the cross validation

In [ ]:
n_folds = 3
kfold = KFold(n_splits=n_folds, random_state=seed,shuffle=True)

### Print the cross validated results of:


1.   Cross validated train 
2.   Cross validated train_s
3.   train on test 
4.   train_s on test








In [ ]:
pipeline

In [ ]:
list_df=[]
for name, models in pipeline:

    # Train score mean 
    cv_results_train = cross_val_score(models, X_train, y_train, cv= kfold, scoring='r2')
    Train_score_mean= cv_results_train.mean()
    # Train score standard deviation 
    Train_score_std= cv_results_train.std()
    # Test score 
    models.fit(X_train, y_train)
    Test= models.score(X_test, y_test)
    
    # Train_s score mean
    cv_results_train_s = cross_val_score(models, X_train_s, y_train_s, cv= kfold, scoring='r2')
    Train_s_score_mean= cv_results_train_s.mean()
    # Train_s standard deviation
    Train_s_score_std= cv_results_train_s.std()
    # Test_s score 
    models.fit(X_train_s, y_train_s)
    Test_s= models.score(X_test, y_test)

    list_df.append([name,Train_score_mean, Train_score_std, Test, Train_s_score_mean, Train_s_score_std, Test_s])

In [ ]:
Scores=pd.DataFrame(list_df, columns=["Model_Name", "Train_score_Mean", "Train_score_STD", "Train_score_Test", "Train_synthetic_Mean", "Train_synthetic_STD", "Train_synthetic_score_Test"])

In [ ]:
Scores